This is an early version that started as a colab notebook, checked in for history only.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Subjects

In [ ]:
SUBJECTS = [
  "/content/drive/MyDrive/projects/tracy/samples/2021-8-19.png",
  "/content/drive/MyDrive/projects/tracy/samples/2021-9-11.png",
  "/content/drive/MyDrive/projects/tracy/samples/2023-1-20.png",
  "/content/drive/MyDrive/projects/tracy/samples/female-1-a.png",
  "/content/drive/MyDrive/projects/tracy/samples/female-1-b.png",
  "/content/drive/MyDrive/projects/tracy/samples/female-2-b.png",
  "/content/drive/MyDrive/projects/tracy/samples/female-3-b.png",
  "/content/drive/MyDrive/projects/tracy/samples/female-4.png",
  "/content/drive/MyDrive/projects/tracy/samples/female-5.png",
  "/content/drive/MyDrive/projects/tracy/samples/female-6.png",
  "/content/drive/MyDrive/projects/tracy/samples/female-7.png",
  "/content/drive/MyDrive/projects/tracy/samples/female-9.png",
  "/content/drive/MyDrive/projects/tracy/samples/jeff-1.png",
  "/content/drive/MyDrive/projects/tracy/samples/male-1-b.png",
  "/content/drive/MyDrive/projects/tracy/samples/male-2-a.png",
  "/content/drive/MyDrive/projects/tracy/samples/male-2-b.png",
  "/content/drive/MyDrive/projects/tracy/samples/female-8-b.png",
  "/content/drive/MyDrive/projects/tracy/samples/female-8-a.png"

]

# Install

In [ ]:
%cd /content
!wget http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_5_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

# Clone and setup the repository
!git clone https://github.com/andresprados/SPIGA.git
%cd SPIGA/
!pip install -e .
#%cd /content


# Image analysis

In [ ]:
import cv2
import numpy as np
from PIL import Image
import dlib
import os
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import math
import json
from google.colab.patches import cv2_imshow
from spiga.inference.config import ModelConfig
from spiga.inference.framework import SPIGAFramework
import copy
from spiga.demo.visualize.plotter import Plotter

# oral commissure
# medial canthus


COLORS = np.array([[255,255,255],[255,0,0],[0,255,0],[0,0,255],[0,255,255],[255,0,255],[255,255,0],[0,0,0]],dtype=np.uint8)
CLUST_NUM = 3
USE_HSV = False

def display_lowres(img, title=""):
  plt.imshow(img)
  plt.title(title)
  plt.show()

def display_image(data, scale):
  img = Image.fromarray(data, 'RGB')
  width, height = img.size

  img = img.resize((
        int(width * scale),
        int(height * scale)),
        Image.ANTIALIAS)
  display(img)

def load_image(filename, crop=True):
  image = cv2.imread(filename)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  face = AnalyzeFace()
  face.load_image(image)
  if crop: face.crop_stylegan()
  return face

class ImageAnalysis:
  def load_image(self, img):
    self.original_img = img.copy()
    self.gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    self.render_img = img.copy()
    self.original_hsv = cv2.cvtColor(self.original_img, cv2.COLOR_RGB2HSV).astype(np.int64)
    self.shape = self.original_img.shape

    self.text_font=cv2.FONT_HERSHEY_SIMPLEX
    self.text_size=0.75
    self.text_color=(255,255,255)
    self.text_thick=1
    self.text_back=5

    self.stats_right = 750

  def write_text(self, pos, txt,color=None, size = 1, thick = 1):
    size = self.text_size * size
    thick = int(self.text_thick * thick)
    if color is None: color = self.text_color
    cv2.putText(self.render_img, txt, pos, self.text_font,
                    size, (0,0,0), thick+self.text_back, cv2.LINE_AA)

    cv2.putText(self.render_img, txt, pos, self.text_font,
                    size, self.text_color, thick, cv2.LINE_AA)

  def write_text_sq(self, pos, txt,color=None):
    if color is None: color = self.text_color

    cv2.putText(self.render_img, txt, pos, self.text_font,
                    self.text_size, (0,0,0), self.text_thick+self.text_back, cv2.LINE_AA)
    w1 = cv2.getTextSize(txt, self.text_font, self.text_size, self.text_thick+self.text_back)[0][0]

    cv2.putText(self.render_img, txt, pos, self.text_font,
                    self.text_size, self.text_color, self.text_thick, cv2.LINE_AA)
    w2 = cv2.getTextSize(txt, self.text_font, self.text_size, self.text_thick)[0][0]



    cv2.putText(self.render_img, "2", (pos[0]+w1-5,pos[1]-5), self.text_font,
                    self.text_size, (0,0,0), self.text_thick+self.text_back, cv2.LINE_AA)

    cv2.putText(self.render_img, "2", (pos[0]+w1-5,pos[1]-5), self.text_font,
                    self.text_size, self.text_color, self.text_thick, cv2.LINE_AA)

  def hline(self, y, x1=None, x2=None, color=(0,255,255),width=5):
    if not x1: x1 = 0
    if not x2: x2 = self.render_img.shape[0]
    cv2.line(self.render_img, (x1,y), (x2,y), color, width)

  def vline(self, x, y1=None, y2=None, color=(0,255,255),width=5):
    if not y1: y1 = 0
    if not y2: y2 = self.render_img.shape[1]
    cv2.line(self.render_img, (x,y1), (x,y2), color, width)

  def line(self, pt1, pt2, color=(0,255,255),width=5):
    cv2.line(self.render_img, pt1, pt2, color, width)

  def circle(self, pt, color=(0,0,255),radius=None):
    if radius is None: radius = int(self.render_img.shape[0] // 200)
    cv2.circle(self.render_img, pt, radius, color, -1)

  def render_reset(self):
    self.render_img = self.original_img.copy()

  def extract_horiz(self, y, x1=None, x2=None):
    if not x1: x1 = 0
    if not x2: x2 = self.render_img.shape[0]
    return self.original_img[y,x1:x2]

  def extract_horiz_hsv(self, y, x1=None, x2=None):
    if not x1: x1 = 0
    if not x2: x2 = self.render_img.shape[0]
    return self.original_hsv[y,x1:x2]

  def save(self, filename):
    image = cv2.cvtColor(self.render_img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(filename,image)

  def measure_polygon(self, contours, alpha=0.4, color=(0,0,255)):
    contours = np.array(contours)
    overlay = self.render_img.copy()
    cv2.fillPoly(overlay, pts = [contours], color =color)
    self.render_img = cv2.addWeighted(overlay, alpha, self.render_img, 1 - alpha, 0)
    contours = contours*face.pix2mm
    x = contours[:,0]
    y = contours[:,1]
    return PolyArea(x,y)

  def line(self, pt1, pt2, color=(255,0,0), thickness=3):
    cv2.line(self.render_img, pt1, pt2, color, thickness)

  # https://www.codeguru.com/multimedia/drawing-an-arrowline/
  def arrow_head(self, pt1, pt2, par=15):
    slopy = math.atan2( ( pt1[1] - pt2[1] ), ( pt1[0] - pt2[0] ) )
    cosy = math.cos( slopy )
    siny = math.sin( slopy )

    self.line(pt1, (pt1[0] + int( - par * cosy - ( par / 2.0 * siny ) ),
      pt1[1] + int( - par * siny + ( par / 2.0 * cosy ) ) ))
    self.line(pt1, (pt1[0] + int( - par * cosy + ( par / 2.0 * siny ) ),
      pt1[1] - int( par / 2.0 * cosy + par * siny ) ) )

  def arrow(self, pt1, pt2, color=(255,0,0), thickness=3, apt1=True, apt2=True):
    self.line(pt1, pt2, color, thickness)

    if apt1:
      self.arrow_head(pt1, pt2)

    if apt2:
      self.arrow_head(pt2, pt1)

# Facial Analysis

In [ ]:
import cv2
import numpy as np
from PIL import Image
import dlib
from matplotlib import pyplot as plt

# Shoelace, https://stackoverflow.com/questions/24467972/calculate-area-of-polygon-given-x-y-coordinates
def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

class StyleGANCrop:
  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor('/content/shape_predictor_5_face_landmarks.dat')

  def find_eyes(self, img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects = StyleGANCrop.detector(gray, 0)

    if len(rects) == 0:
      raise ValueError("No faces detected")
    elif len(rects) > 1:
      raise ValueError("Multiple faces detected")

    shape = StyleGANCrop.predictor(gray, rects[0])
    features = []

    for i in range(0, 5):
      features.append((i, (shape.part(i).x, shape.part(i).y)))

    return (int(features[3][1][0] + features[2][1][0]) // 2, \
      int(features[3][1][1] + features[2][1][1]) // 2), \
      (int(features[1][1][0] + features[0][1][0]) // 2, \
      int(features[1][1][1] + features[0][1][1]) // 2)


  def measure_stylegan(self, img):
    left_eye, right_eye = self.find_eyes(img)
    d = abs(right_eye[0] - left_eye[0])
    z = 255/d
    ar = img.shape[0]/img.shape[1]
    w = img.shape[1] * z
    img2 = cv2.resize(img, (int(w), int(w*ar)))
    bordersize = 1024
    img3 = cv2.copyMakeBorder(
        img2,
        top=bordersize,
        bottom=bordersize,
        left=bordersize,
        right=bordersize,
        borderType=cv2.BORDER_CONSTANT,value=(0,0,0))

    left_eye2, right_eye2 = self.find_eyes(img3)

    crop1 = left_eye2[0] - 385
    crop0 = left_eye2[1] - 490

    return [(int(w), int(w*ar)), crop0, crop1]

  def size_crop(self, img, sz, crop0, crop1):
    img2 = cv2.resize(img, sz)
    bordersize = 1024
    img3 = cv2.copyMakeBorder(
        img2,
        top=bordersize,
        bottom=bordersize,
        left=bordersize,
        right=bordersize,
        borderType=cv2.BORDER_CONSTANT,value=(0,0,0))
    return img3[crop0:crop0+1024,crop1:crop1+1024]


  def crop_stylegan(self, img):
    left_eye, right_eye = self.find_eyes(img)
    d = abs(right_eye[0] - left_eye[0])
    z = 255/d
    ar = img.shape[0]/img.shape[1]
    w = img.shape[1] * z
    img2 = cv2.resize(img, (int(w), int(w*ar)))
    bordersize = 1024
    img3 = cv2.copyMakeBorder(
        img2,
        top=bordersize,
        bottom=bordersize,
        left=bordersize,
        right=bordersize,
        borderType=cv2.BORDER_CONSTANT,value=(0,0,0))

    left_eye2, right_eye2 = self.find_eyes(img3)

    crop1 = left_eye2[0] - 385
    crop0 = left_eye2[1] - 490
    return img3[crop0:crop0+1024,crop1:crop1+1024]

class AnalyzeFace (ImageAnalysis):
  crop = StyleGANCrop()
  processor = SPIGAFramework(ModelConfig('wflw'))

  def __init__(self):
    self.left_eye = None
    self.right_eye = None
    self.nose = None

  def _find_landmarks(self, img):

    bbox = [0,0,img.shape[1],img.shape[0]]
    features = AnalyzeFace.processor.inference(img, [bbox])

    # Prepare variables
    x0,y0,w,h = bbox
    landmarks2 = [(int(x[0]),int(x[1])) for x in np.array(features['landmarks'][0])]
    return landmarks2

    #landmarks = []
    #for i in range(68):
    #  landmarks.append([int(shape.part(i).x),int(shape.part(i).y)])
    #return landmarks

  def load_image(self, img):
    super().load_image(img)
    self.landmarks = self._find_landmarks(img)

    # 45 = left edge of left eye
    # 42 = right edge of left eye
    # 36 = right edge of right eye
    # 39 = left edge of right eye
    # 33 = nose (bottom center)

    self.right_eye = (int(self.landmarks[60][0] + self.landmarks[64][0]) // 2, \
      int(self.landmarks[60][1] + self.landmarks[64][1]) // 2)
    self.left_eye = (int(self.landmarks[68][0] + self.landmarks[72][0]) // 2, \
      int(self.landmarks[68][1] + self.landmarks[72][1]) // 2)

    self.nose = self.landmarks[57]
    self._estimate_mouth()
    self.pupillary_distance = abs(self.right_eye[0] - self.left_eye[0])
    self.pix2mm = 63/self.pupillary_distance

  def crop_stylegan(self):
    img = self.crop.crop_stylegan(self.original_img)
    self.load_image(img)

  def _estimate_mouth(self):
    x1 = int(1e50)
    y1 = int(1e50)
    x2 = 0
    y2 = 0
    for i in range(48,68):
      landmark = self.landmarks[i]
      x1 = min(x1,landmark[0])
      y1 = min(y1,landmark[1]-25)
      x2 = max(x2,landmark[0])
      y2 = max(y2,landmark[1]+25)

    self.mouth_p1 = (x1,y1)
    self.mouth_p2 = (x2,y2)

  def scan_vert(self, y):
    y+=self.nose[1]
    x1 = self.nose[0] - (self.mouth_width//2)
    x2 = self.nose[0] + (self.mouth_width//2)
    scan_line_color = self.original_hsv[y,x1:x2]
    scan_line = scan_line_color.mean(axis=1).astype(int)
    scan_line2 = np.append(scan_line[0] , scan_line[:-1])
    scan_diff = np.absolute(scan_line - scan_line2)
    white_line = np.repeat(255,self.mouth_width*3).reshape((self.mouth_width,3))
    teeth_pred = np.sqrt(((white_line - scan_line_color)**2).sum(axis=1)).astype(int)
    return scan_line_color #, teeth_pred

  def mark_teeth(self, y, teeth_pred):
    x1 = self.nose[0] - (self.mouth_width//2)
    y+=self.nose[1]
    for i in range(len(teeth_pred)):
      if teeth_pred[i]<150:
        self.render_img[y,x1+i] = (0,0,255)

  def draw_landmarks(self, size=0.25, color=[0,255,255],numbers=False):
    for i,landmark in enumerate(self.landmarks):
      self.circle(landmark,radius=2,color=color)
      if numbers:
        self.write_text(landmark,str(i))
    self.circle(self.left_eye, color=color)
    self.circle(self.right_eye, color=color)


  def cluster_mouth_rect(self):
    x1, y1 = self.mouth_p1
    x2, y2 = self.mouth_p2

    if USE_HSV:
      mouth_img = self.original_hsv[y1:y2,x1:x2]
    else:
      mouth_img = self.original_img[y1:y2,x1:x2]

    mouth_pixels = mouth_img.reshape(mouth_img.shape[0]*mouth_img.shape[1],3)
    return KMeans(n_clusters=CLUST_NUM, random_state=42, n_init=1, init='k-means++').fit(mouth_pixels)


  def segment_line(self, kmeans, y):
    x1, y1 = self.mouth_p1
    x2, y2 = self.mouth_p2
    lst = []

    if USE_HSV:
      l = self.extract_horiz_hsv(y)
    else:
      l = self.extract_horiz(y)
    return kmeans.predict(l[x1:x2])

  def segment_display_line(self, kmeans, y):
    labels = self.segment_line(kmeans, y)

    lst = []
    for v in labels:
      lst.append(COLORS[v])
    l2 = np.array(lst)
    l2 = l2.reshape((1,l2.shape[0],l2.shape[1]))

    x1, y1 = self.mouth_p1
    x2, y2 = self.mouth_p2

    lfill = np.repeat(COLORS[-1].reshape((1,3)),x1,axis=0)
    lfill = lfill.reshape([1,lfill.shape[0],lfill.shape[1]])
    l2 = np.concatenate([lfill,l2],axis=1)

    l2 = np.repeat(l2,50,axis=0)
    return l2


  def stretch_scan_line(self, l):
    x1, y1 = self.mouth_p1
    x2, y2 = self.mouth_p2
    lst = []

    l2 = l.copy()
    l2[:x1,:] = 0
    l2[x2:,:] = 0
    l2 = l2.reshape((1,l2.shape[0],l2.shape[1]))
    l2 = np.repeat(l2,50,axis=0)
    return l2

  def scan_mouth(self, kmeans):
    x1, y1 = self.mouth_p1
    x2, y2 = self.mouth_p2

    if USE_HSV:
      mouth_img = self.original_hsv[y1:y2,x1:x2]
    else:
      mouth_img = self.original_img[y1:y2,x1:x2]

    for y in range(len(mouth_img)):
      labels = kmeans.predict(mouth_img[y])
      lst = []
      for v in labels:
        lst.append(COLORS[v])
      l2 = np.array(lst)
      l2 = l2.reshape((1,l2.shape[0],l2.shape[1]))
      mouth_img[y] = l2

    return mouth_img

  def mark_mouth(self, color=(255,0,0), thickness=3):
    cv2.rectangle(self.render_img, self.mouth_p1, self.mouth_p2, color, thickness)

  def measure(self, pt1, pt2, color=(255,0,0), thickness=3):
    self.arrow(pt1, pt2, color, thickness)
    d = math.dist(pt1,pt2) * self.pix2mm
    mp = [int((pt1[0]+pt2[0])//2),int((pt1[1]+pt2[1])//2)]
    self.write_text(mp,f"{d:.2f}mm")
    return d

  def dental_area_initial(self):
      # Mark inner mouth area as found by neural network
      z = np.zeros( self.original_img.shape, dtype=np.uint8)
      contours = [self.landmarks[88],
                  self.landmarks[89],
                  self.landmarks[90],
                  self.landmarks[91],
                  self.landmarks[92],
                  self.landmarks[93],
                  self.landmarks[94],
                  self.landmarks[95]]

      contours = np.array(contours)
      cv2.fillPoly(z, pts = [contours], color =(255,255,255))

      # Extract vertical mouth scan lines
      dental = []
      for x in range(self.original_img.shape[0]):
        s = z[:,x].sum()
        if s:
          s = np.where(z[:,x,0])
          dental.append( {'x':x, 'y1': min(s[0]), 'y2':max(s[0])} )
      return dental

  def measure_dental_area(self):
    contours = [self.landmarks[88],
              self.landmarks[89],
              self.landmarks[90],
              self.landmarks[91],
              self.landmarks[92],
              self.landmarks[93],
              self.landmarks[94],
              self.landmarks[95]]

    contours = np.array(contours)
    contours = contours*self.pix2mm
    x = contours[:,0]
    y = contours[:,1]
    return PolyArea(x,y)

  def measure_eye_area(self):
    right_eye_area = self.measure_polygon(
      [self.landmarks[60],
      self.landmarks[61],
      self.landmarks[62],
      self.landmarks[63],
      self.landmarks[64],
      self.landmarks[65],
      self.landmarks[66],
      self.landmarks[67]])

    left_eye_area = self.measure_polygon(
      [self.landmarks[68],
      self.landmarks[69],
      self.landmarks[70],
      self.landmarks[71],
      self.landmarks[72],
      self.landmarks[73],
      self.landmarks[74],
      self.landmarks[75]])

    eye_area_diff = abs(right_eye_area-left_eye_area)
    #66,74

    self.write_text_sq((self.landmarks[66][0]-50,self.landmarks[66][1]+20),f"R={round(right_eye_area,2)}mm")
    self.write_text_sq((self.landmarks[74][0]-50,self.landmarks[74][1]+20),f"L={round(left_eye_area,2)}mm")
    self.write_text_sq((self.stats_right,self.landmarks[74][1]+50),f"d.eyes={round(eye_area_diff,2)}mm")
    return left_eye_area, right_eye_area, eye_area_diff

    #self.write_text((10, 950),f"L={round(left_eye_area,2)}mm^2, R={round(right_eye_area,2)}mm^2, dif={round(eye_area_diff)}")
    #self.write_text((10, 950),f"Eyes: L={round(left_eye_area,2)}mm^2, R={round(right_eye_area,2)}mm^2, dif={round(eye_area_diff)}")
    #self.write_text((10, 950),f"Eyes: L={round(left_eye_area,2)}mm^2, R={round(right_eye_area,2)}mm^2, dif={round(eye_area_diff)}")

  def measure_brows(self):
    # left brow
    contours = [self.landmarks[34],
              self.landmarks[35],
              self.landmarks[36],
              self.landmarks[37]]

    contours = np.array(contours)
    x = contours[:,0]
    y = contours[:,1]
    left_brow_idx = np.argmin(y)
    left_brow_y = y[left_brow_idx]
    left_brow_x = x[left_brow_idx]
    self.arrow((left_brow_x, left_brow_y), (1024,left_brow_y),apt2=False)

    # right brow
    contours = [self.landmarks[42],
              self.landmarks[43],
              self.landmarks[44],
              self.landmarks[45]]

    contours = np.array(contours)
    x = contours[:,0]
    y = contours[:,1]
    right_brow_idx = np.argmin(y)
    right_brow_y = y[right_brow_idx]
    right_brow_x = x[right_brow_idx]
    self.arrow((right_brow_x, right_brow_y), (1024,right_brow_y),apt2=False)

    diff = abs(left_brow_y-right_brow_y)*self.pix2mm
    self.write_text((self.stats_right, min(left_brow_y,right_brow_y)-10),f"d.brow={diff:.2f} mm")
    return diff


  def analyze(self):
    d1 = self.measure(self.landmarks[64],self.landmarks[76])
    d2 = self.measure(self.landmarks[68],self.landmarks[82])
    if d1>d2:
      fai = d1 - d2
    else:
      fai = d2 - d1

    dental_sqmm = self.measure_dental_area()

    fai_pt = (self.stats_right,self.landmarks[82][1])
    self.write_text(fai_pt, f"FAI={fai:.2f} mm")
    dental_pt = (self.stats_right,self.landmarks[85][1]+20)
    self.write_text_sq(dental_pt,f"dental={round(dental_sqmm,2)}mm")
    dental_area = self.dental_area_initial()

    for scan in dental_area:
      x, y1, y2 = scan['x'], scan['y1'], scan['y2']
      for y in range(y1,y2):
        self.render_img[y][x][2] = 255

    left_eye_area, right_eye_area, eye_area_diff = self.measure_eye_area()
    brow_diff = self.measure_brows()
    return {'fai': fai, 'left_eye_area':left_eye_area, 'right_eye_area':right_eye_area, 'eye_area_diff':eye_area_diff, 'brow_diff':brow_diff}

# Test Single

In [ ]:
#subject = "/content/drive/MyDrive/projects/tracy/samples/after.jpg"
subject = "/content/drive/MyDrive/projects/tracy/samples/2021-8-19.png"
#subject = "/content/drive/MyDrive/projects/tracy/samples/mark_ruffalo.jpg"
# 60 - 67
face = load_image(subject)
face.analyze()
face.draw_landmarks(numbers=True)
print(subject)
display_image(face.render_img, scale=1)

# Test All Images

In [ ]:
import os

OUTPUT = "/content/drive/MyDrive/projects/tracy/samples-out/"

for idx, subject in enumerate(SUBJECTS):
  face = load_image(subject)
  face.analyze()
  #face.draw_landmarks(numbers=True)
  face.save(os.path.join(OUTPUT,f"{idx}.jpg"))
  print(subject)
  display_image(face.render_img, scale=1)

# Video

In [ ]:
from os import listdir
from os.path import isfile, join
import tqdm
from datetime import datetime

PATH_INPUT = '/content/drive/MyDrive/projects/tracy/orig'
PATH_OUTPUT = '/content/drive/MyDrive/projects/tracy/output'

files = [f for f in listdir(PATH_INPUT) if isfile(join(PATH_INPUT, f))]
files.sort()

def sort_filenames(filenames):
    def extract_date(filename):
        # Extract the date string from the filename
        date_string = filename.split(".")[0].split("-")
        year, month, day = date_string[-3:]
        date_string = year + month.zfill(2) + day.zfill(2)
        # Return the date string in the format "YYYYMMDD"
        return date_string

    # Sort the filenames using the custom key function extract_date
    sorted_filenames = sorted(filenames, key=extract_date)
    return sorted_filenames

files = sort_filenames(files)

In [ ]:
id = 1

lst_fai = []
lst_date = []

for fn in tqdm.tqdm(files):
  try:
    face = load_image(os.path.join(PATH_INPUT, fn))
  except:
    continue
  fai = face.analyze()
  date_string = fn.split(".")[0]
  date = datetime.strptime(date_string, "%Y-%m-%d")
  formatted_date = str(date.strftime("%Y, %B %d"))
  face.write_text((10,40),formatted_date,color=(255,255,0),size=2,thick=2)
  #write_text(cropped_image2, formatted_date)
  for z in range(5):
    ofn = os.path.join(PATH_OUTPUT, f"{id}.png")
    face.save(ofn)
    #print(ofn)
    #lst_fai.append(fai)
    #lst_date.append(date)
    id+=1
  #except:
    #pass

  #display_lowres(face.render_img)
  #cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

In [ ]:
!ffmpeg -r 30 -i /content/drive/MyDrive/projects/tracy/output/%d.png -vcodec mpeg4 -y /content/drive/MyDrive/projects/tracy/movie.mp4

In [ ]:
#!rm /content/drive/MyDrive/projects/tracy/output/*
!ls /content/drive/MyDrive/projects/tracy/output

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

layout = go.Layout(
    #title="FAI",
    autosize=False,
    width=960,
    height=540,
    xaxis_title="Time",
    xaxis_showticklabels = True,
    yaxis_title="FAI",
    yaxis_showticklabels = True)

fig = go.Figure(layout=layout)

fig.add_trace(go.Scatter(x=lst_date, y=lst_fai, fill='tozeroy', name="Positive", line=dict(color='red')))

fig.show()

In [ ]:
lst_fai

# Blink

ffmpeg -i IMG_4390_1.mp4 -vf fps=30 tracy-intent-blink%d.png


In [ ]:
import os

INPUT = "/content/drive/MyDrive/projects/tracy/blink/test/"
OUTPUT = "/content/drive/MyDrive/projects/tracy/output/"

!rm {OUTPUT}/*

RATE = 1/240
print(RATE)

min_fai = 1000
min_left_eye_area = 1000
min_right_eye_area = 1000
min_eye_area_diff = 1000
min_brow_diff = 1000
max_fai = 0
max_left_eye_area = 0
max_right_eye_area = 0
max_eye_area_diff = 0
max_brow_diff = 0

PREFIX = "img"
# sample 1st
filename = os.path.join(INPUT,f"{PREFIX}1.png")
print(filename)
crop = StyleGANCrop()
image = cv2.imread(filename)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
face = AnalyzeFace()
face.load_image(image)
sz, crop0, crop1 = crop.measure_stylegan(face.original_img)
print("---")
print(sz)
print(crop0)
print(crop1)

left_area = []
right_area = []

out_idx = 1
idx = 0

for idx in range(269):
  idx+=1
  filename = os.path.join(INPUT,f"{PREFIX}{idx}.png")
  if not os.path.exists(filename): break
  print(filename)
  # Load frame and crop/size
  image = cv2.imread(filename)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  face = AnalyzeFace()
  face.load_image(image)
  face.load_image(crop.size_crop(face.original_img, sz, crop0, crop1))

  # Analyze blink frame
  rec = face.analyze()
  fai = rec['fai']
  left_eye_area = rec['left_eye_area']
  right_eye_area = rec['right_eye_area']
  eye_area_diff = rec['eye_area_diff']
  brow_diff = rec['brow_diff']

  left_area.append(float(left_eye_area))
  right_area.append(float(right_eye_area))

  min_fai = min(min_fai,fai)
  min_left_eye_area = min(min_left_eye_area,left_eye_area)
  min_right_eye_area = min(min_right_eye_area,right_eye_area)
  min_eye_area_diff = min(min_eye_area_diff,eye_area_diff)
  min_brow_diff = min(min_brow_diff,brow_diff)
  max_fai = max(max_fai,fai)
  max_left_eye_area = max(max_left_eye_area,left_eye_area)
  max_right_eye_area = max(max_right_eye_area,right_eye_area)
  max_eye_area_diff = max(max_eye_area_diff,eye_area_diff)
  max_brow_diff = max(max_brow_diff,brow_diff)

  face.write_text((10,50), f"Frame {idx+1}, {round(idx*RATE*1000)} ms")
  face.write_text((10,80), f"(min/max) left eye: {round(min_left_eye_area)}/{round(max_left_eye_area)} right eye: {round(min_right_eye_area)}/{round(max_right_eye_area)} diff eye:{round(min_eye_area_diff)}/{round(max_eye_area_diff)}")
  #face.draw_landmarks(numbers=True)
  for i in range(5):
    face.save(os.path.join(OUTPUT,f"blink-{out_idx}.jpg"))
    out_idx += 1
  #display_image(face.render_img, scale=1)

In [ ]:
!ls /content/drive/MyDrive/projects/tracy/blink/test/

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

lst_time = [x*RATE*2000 for x in range(len(left_area))]

layout = go.Layout(
    title=f"Blink Efficency: {PREFIX}",
    autosize=False,
    width=960,
    height=540,
    xaxis_title="Time (ms)",
    xaxis_showticklabels = True,
    yaxis_title="Area (mm^2)",
    yaxis_showticklabels = True)

fig = go.Figure(layout=layout)

fig.add_trace(go.Scatter(x=lst_time, y=left_area, name="Left Area", line=dict(color='red')))
fig.add_trace(go.Scatter(x=lst_time, y=right_area, name="Right Area", line=dict(color='blue')))

fig.show()

In [ ]:
!ffmpeg -r 30 -i /content/drive/MyDrive/projects/tracy/output/blink-%d.jpg -vcodec mpeg4 -y /content/drive/MyDrive/projects/tracy/temp.mp4

In [ ]:
import cv2
import numpy as np

filename = "/content/drive/MyDrive/projects/tracy/samples/2021-8-19.png"
img = cv2.imread(filename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
original_img = img.copy()
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
render_img = img.copy()
original_hsv = cv2.cvtColor(original_img, cv2.COLOR_RGB2HSV).astype(np.int64)
shape = original_img.shape

In [ ]:
shape